<a href="https://colab.research.google.com/github/nkinsman16/Homework-Week-4/blob/main/Homework_week_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# list.files() returns the names of files and folders in my directory.
#Important to ensure I have uploaded the previous three documents from HW #3.
list.files()

[1] "infant_mortality_clean.csv"   "Life_Expectancy_clean.csv"   
[3] "maternal_mortality_clean.csv" "sample_data"

In [3]:
# The library(tidyverse) is a collection of R packages for data manipulation, visualization and analysis.
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.6
✔ forcats   1.0.1     ✔ stringr   1.6.0
✔ ggplot2   4.0.1     ✔ tibble    3.3.0
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.2.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [4]:
# We are reading all three documents using the read_csv() and store them as data frames and renaming them.
infant <- read_csv("infant_mortality_clean.csv") # Recallign the document infant
maternal <- read_csv("maternal_mortality_clean.csv") # Recalling the document maternal
life <- read_csv("Life_Expectancy_clean.csv") #realling the document life

# Verifying the column names to ensure the data is correctly show and prevent any errors.
colnames(infant)
colnames(maternal)
colnames(life)


Rows: 227 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): country, region
dbl (1): variable

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 196 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): country, region
dbl (1): variable

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.
Rows: 227 Columns: 3
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): country, region
dbl (1): variable

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


[1] "country"  "variable" "region"

[1] "country"  "variable" "region"

[1] "country"  "variable" "region"

In [5]:
#Need to rename the data sets to ensure that the various variable columns are separated as we can't join them and we need them as individual columns.
infant <- infant %>% # Infant is the data frame being updated and reassigned the <- operator saves the modified version back to the same object. %>% pipe passes the data frame forward to the next function.
  rename(infant_mortality = variable) # Uses rename() to change the column variable to infant_mortality

maternal <- maternal %>% #Taking the maternal dataset and renaming the variable to maternal_mortality
  rename(maternal_mortality = variable) # Renaming variable in the maternal dataset to maternal_mortality

life <- life %>% # Taking the life dataset and renaming the variable column to life_expectancy
  rename(life_expectancy = variable) # Renaming the variable in the life dataset to life_expectancy


In [15]:
# Creating a new dataset called merged_LMI, which stores the dataset infant as the main dataset, the %>% pipe passes it step by step through each join.
merged_LMI <- infant %>%
  left_join(maternal, by = c("country", "region")) %>% #Leftjoin() merges datasets while keeping all rows from infant, maternal and life datasets. Using Country and Region to match the rows across the datasets.
  left_join(life, by = c("country", "region"))   # Using Country and Region to merge the rows together.

# Need to confirm the joins were successful.
head(merged_LMI)


country,infant_mortality,region,maternal_mortality,life_expectancy
<chr>,<dbl>,<chr>,<dbl>,<dbl>
Somalia,81.5,Africa,563,56.5
Central African Republic,79.3,Africa,692,56.4
Equatorial Guinea,76.9,Africa,174,63.9
Sierra Leone,70.1,Africa,354,59.4
Nigeria,65.6,Africa,993,62.2
Niger,63.0,Africa,350,60.9


In [16]:
# I wanted to reorder the columns so that country and region are together.
merged_LMI <- merged_LMI %>%
  select(country, region, infant_mortality, maternal_mortality, life_expectancy)


In [19]:
# Confirming the column order
colnames(merged_LMI)


[1] "country"            "region"             "infant_mortality"  
[4] "maternal_mortality" "life_expectancy"

In [20]:
#Creating a CSV for my merged content
write_csv(merged_LMI, "merged_LMI.csv")


In [30]:
# Aggregate a aggregated file using region
merged_LMI_region_agg <- merged_LMI %>%
  group_by(region) %>%   #Using the group by region to organizae the countries into regions.
  summarise( # Applying the summarise() to calculate the mean of the various columns, Infant_mortality, maternal_mortality, and life_expectancy
    mean_infant_mortality = mean(infant_mortality, na.rm = TRUE),  # Creates a new variable called mean_infant_mortality by using the mean() function to calculate the average, using na.rm = TRUE to removes missingh values, to ensure they don't distort the result.
    mean_maternal_mortality = mean(maternal_mortality, na.rm = TRUE), #Create a new variable called mean_maternal_mortality
    mean_life_expectancy = mean(life_expectancy, na.rm = TRUE) #Create a new variable called mean_life_expectancy
  )


In [23]:
# Save aggregated dataset
write_csv(merged_LMI_region_agg, "merged_LMI_region_aggregated.csv")

# Verify file creation
file.exists("merged_LMI_region_aggregated.csv")


[1] TRUE

In [26]:
#Converting our orginal merged dataset into the long version using the pivot_longer()
merged_LMI_long <- merged_LMI %>% # Creating the variable merged_LMI_long
  pivot_longer( # Pivot_longer() is a functin that reshapes data from wide format to long format by turning multiple columns into key-value pairs. Long format data easier to analyze, summarize and visualize
    cols = c(infant_mortality, maternal_mortality, life_expectancy), #Specifies the three health variables to be transformed
    names_to = "indicator", # names_to = "indictor" stores the the names as new categorical variables
    values_to = "value"   # values_to = "values" stores the corresponding
  )


In [28]:
# Need to confirm the long format is successful.
head(merged_LMI_long)


country,region,indicator,value
<chr>,<chr>,<chr>,<dbl>
Somalia,Africa,infant_mortality,81.5
Somalia,Africa,maternal_mortality,563.0
Somalia,Africa,life_expectancy,56.5
Central African Republic,Africa,infant_mortality,79.3
Central African Republic,Africa,maternal_mortality,692.0
Central African Republic,Africa,life_expectancy,56.4


In [29]:
# Save long-format dataset
write_csv(merged_LMI_long, "merged_LMI_long.csv")

# Verify file creation
file.exists("merged_LMI_long.csv")


[1] TRUE